In [5]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def detect_features(image, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create()
    elif algorithm == 'ORB':
        detector = cv2.ORB_create(nfeatures=1000)
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create()
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create()
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")

    keypoints, descriptors = detector.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2, algorithm):
    if descriptors1 is None or descriptors2 is None:
        return []

    if algorithm in ['SIFT', 'AKAZE', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    else:  # ORB
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)
    return sorted(matches, key=lambda x: x.distance)

def evaluate_feature_quality(matches, num_keypoints):
    if not matches or num_keypoints == 0:
        return 0, 0, 0

    distances = [m.distance for m in matches]
    avg_distance = np.mean(distances)
    min_distance = np.min(distances)
    max_distance = np.max(distances)

    # Adjust the threshold for "good" matches
    good_matches = [m for m in matches if m.distance < 0.8 * avg_distance]
    quality = len(good_matches) / num_keypoints

    return quality, avg_distance, len(good_matches)

def evaluate_repeatability(keypoints1, keypoints2, matches, shape):
    if len(matches) < 4:
        return 0

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    try:
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if H is None:
            return 0

        h, w = shape
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, H)

        good_matches = mask.ravel() == 1
        return np.sum(good_matches) / len(matches)

    except cv2.error:
        return 0

def evaluate_algorithm_extended(algorithm, image_folder):
    image_files = sorted(os.listdir(image_folder))
    quality_scores = []
    repeatability_scores = []
    total_keypoints = []
    match_counts = []
    avg_distances = []
    good_match_counts = []

    for i in tqdm(range(len(image_files) - 1), desc=f"Evaluating {algorithm}"):
        image1_path = os.path.join(image_folder, image_files[i])
        image2_path = os.path.join(image_folder, image_files[i+1])

        image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

        if image1 is None or image2 is None:
            print(f"Warning: Unable to read image {image_files[i]} or {image_files[i+1]}")
            continue

        keypoints1, descriptors1 = detect_features(image1, algorithm)
        keypoints2, descriptors2 = detect_features(image2, algorithm)

        total_keypoints.append(len(keypoints1))

        matches = match_features(descriptors1, descriptors2, algorithm)
        match_counts.append(len(matches))

        quality_score, avg_distance, good_match_count = evaluate_feature_quality(matches, len(keypoints1))
        quality_scores.append(quality_score)
        avg_distances.append(avg_distance)
        good_match_counts.append(good_match_count)

        repeatability_score = evaluate_repeatability(keypoints1, keypoints2, matches, image1.shape)
        repeatability_scores.append(repeatability_score)

    avg_keypoints = np.mean(total_keypoints) if total_keypoints else 0
    avg_matches = np.mean(match_counts) if match_counts else 0
    avg_quality = np.mean(quality_scores) if quality_scores else 0
    avg_repeatability = np.mean(repeatability_scores) if repeatability_scores else 0
    avg_distance = np.mean(avg_distances) if avg_distances else 0
    avg_good_matches = np.mean(good_match_counts) if good_match_counts else 0

    return avg_quality, avg_repeatability, avg_keypoints, avg_matches, avg_distance, avg_good_matches

# 메인 실행 부분
image_folder = "/content/drive/MyDrive/KeyPoint/dataset_split/test/damaged"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

extended_results = {}

for algorithm in algorithms:
    quality, repeatability, avg_keypoints, avg_matches, avg_distance, avg_good_matches = evaluate_algorithm_extended(algorithm, image_folder)
    extended_results[algorithm] = {
        'quality': quality,
        'repeatability': repeatability,
        'avg_keypoints': avg_keypoints,
        'avg_matches': avg_matches,
        'avg_distance': avg_distance,
        'avg_good_matches': avg_good_matches
    }

print("Extended Results:")
for alg, res in extended_results.items():
    print(f"{alg}:")
    print(f"  Quality = {res['quality']:.4f}")
    print(f"  Repeatability = {res['repeatability']:.4f}")
    print(f"  Avg Keypoints = {res['avg_keypoints']:.2f}")
    print(f"  Avg Matches = {res['avg_matches']:.2f}")
    print(f"  Avg Match Distance = {res['avg_distance']:.2f}")
    print(f"  Avg Good Matches = {res['avg_good_matches']:.2f}")
    print()

Evaluating BRISK: 100%|██████████| 59/59 [00:50<00:00,  1.17it/s]

Extended Results:
SIFT:
  Quality = 0.0153
  Repeatability = 0.1033
  Avg Keypoints = 341.44
  Avg Matches = 93.05
  Avg Match Distance = 325.71
  Avg Good Matches = 4.32

ORB:
  Quality = 0.0166
  Repeatability = 0.0633
  Avg Keypoints = 739.31
  Avg Matches = 194.44
  Avg Match Distance = 66.11
  Avg Good Matches = 11.46

AKAZE:
  Quality = 0.0092
  Repeatability = 0.1892
  Avg Keypoints = 151.37
  Avg Matches = 44.36
  Avg Match Distance = 637.31
  Avg Good Matches = 0.97

BRISK:
  Quality = 0.0038
  Repeatability = 0.0390
  Avg Keypoints = 2053.53
  Avg Matches = 530.36
  Avg Match Distance = 646.44
  Avg Good Matches = 3.76



In [6]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def detect_features(image, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create(nfeatures=1000)  # 특징점 수 증가
    elif algorithm == 'ORB':
        detector = cv2.ORB_create(nfeatures=1500, scaleFactor=1.2, nlevels=8)  # 파라미터 조정
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create(threshold=0.001)  # 임계값 낮춤
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create(thresh=10, octaves=4)  # 파라미터 조정
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")

    keypoints, descriptors = detector.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2, algorithm):
    if descriptors1 is None or descriptors2 is None:
        return []

    if algorithm in ['SIFT', 'AKAZE', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_L2)
    else:  # ORB
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    # 2-nn 매칭 사용
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:  # Lowe's ratio test
            good_matches.append(m)

    return good_matches

def evaluate_feature_quality(matches, num_keypoints):
    if not matches or num_keypoints == 0:
        return 0, 0, 0

    distances = [m.distance for m in matches]
    avg_distance = np.mean(distances)

    # Quality 계산 방식 변경
    quality = len(matches) / num_keypoints

    return quality, avg_distance, len(matches)

def evaluate_repeatability(keypoints1, keypoints2, matches, shape):
    if len(matches) < 4:
        return 0

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    try:
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if H is None:
            return 0

        h, w = shape
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, H)

        good_matches = mask.ravel() == 1
        return np.sum(good_matches) / len(matches)

    except cv2.error:
        return 0

def preprocess_image(image):
    # 이미지 전처리: 대비 향상
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(image)

def evaluate_algorithm_extended(algorithm, image_folder):
    image_files = sorted(os.listdir(image_folder))
    quality_scores = []
    repeatability_scores = []
    total_keypoints = []
    match_counts = []
    avg_distances = []

    for i in tqdm(range(len(image_files) - 1), desc=f"Evaluating {algorithm}"):
        image1_path = os.path.join(image_folder, image_files[i])
        image2_path = os.path.join(image_folder, image_files[i+1])

        image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

        if image1 is None or image2 is None:
            print(f"Warning: Unable to read image {image_files[i]} or {image_files[i+1]}")
            continue

        # 이미지 전처리 적용
        image1 = preprocess_image(image1)
        image2 = preprocess_image(image2)

        keypoints1, descriptors1 = detect_features(image1, algorithm)
        keypoints2, descriptors2 = detect_features(image2, algorithm)

        total_keypoints.append(len(keypoints1))

        matches = match_features(descriptors1, descriptors2, algorithm)
        match_counts.append(len(matches))

        quality_score, avg_distance, good_match_count = evaluate_feature_quality(matches, len(keypoints1))
        quality_scores.append(quality_score)
        avg_distances.append(avg_distance)

        repeatability_score = evaluate_repeatability(keypoints1, keypoints2, matches, image1.shape)
        repeatability_scores.append(repeatability_score)

    avg_keypoints = np.mean(total_keypoints) if total_keypoints else 0
    avg_matches = np.mean(match_counts) if match_counts else 0
    avg_quality = np.mean(quality_scores) if quality_scores else 0
    avg_repeatability = np.mean(repeatability_scores) if repeatability_scores else 0
    avg_distance = np.mean(avg_distances) if avg_distances else 0

    return avg_quality, avg_repeatability, avg_keypoints, avg_matches, avg_distance

# 메인 실행 부분
image_folder = "/content/drive/MyDrive/KeyPoint/dataset_split/test/damaged"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

extended_results = {}

for algorithm in algorithms:
    quality, repeatability, avg_keypoints, avg_matches, avg_distance = evaluate_algorithm_extended(algorithm, image_folder)
    extended_results[algorithm] = {
        'quality': quality,
        'repeatability': repeatability,
        'avg_keypoints': avg_keypoints,
        'avg_matches': avg_matches,
        'avg_distance': avg_distance
    }

print("Extended Results:")
for alg, res in extended_results.items():
    print(f"{alg}:")
    print(f"  Quality = {res['quality']:.4f}")
    print(f"  Repeatability = {res['repeatability']:.4f}")
    print(f"  Avg Keypoints = {res['avg_keypoints']:.2f}")
    print(f"  Avg Matches = {res['avg_matches']:.2f}")
    print(f"  Avg Match Distance = {res['avg_distance']:.2f}")
    print()

Evaluating BRISK: 100%|██████████| 59/59 [01:51<00:00,  1.89s/it]

Extended Results:
SIFT:
  Quality = 0.0058
  Repeatability = 0.1546
  Avg Keypoints = 496.73
  Avg Matches = 2.61
  Avg Match Distance = 142.13

ORB:
  Quality = 0.0039
  Repeatability = 0.4502
  Avg Keypoints = 1186.39
  Avg Matches = 4.46
  Avg Match Distance = 47.85

AKAZE:
  Quality = 0.0026
  Repeatability = 0.0000
  Avg Keypoints = 336.54
  Avg Matches = 0.85
  Avg Match Distance = 256.59

BRISK:
  Quality = 0.0010
  Repeatability = 0.1978
  Avg Keypoints = 4896.76
  Avg Matches = 3.41
  Avg Match Distance = 360.74



In [7]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def detect_features(image, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create(nfeatures=500, contrastThreshold=0.04)
    elif algorithm == 'ORB':
        detector = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8)
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create(threshold=0.001)
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create(thresh=30, octaves=3)
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")

    keypoints, descriptors = detector.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2, algorithm):
    if descriptors1 is None or descriptors2 is None:
        return []

    if algorithm in ['SIFT', 'AKAZE', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    else:  # ORB
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # 상위 75%의 매치만 사용
    good_matches = matches[:int(len(matches) * 0.75)]

    return good_matches

def evaluate_feature_quality(matches, num_keypoints):
    if not matches or num_keypoints == 0:
        return 0, 0, 0

    distances = [m.distance for m in matches]
    avg_distance = np.mean(distances)

    # Quality 계산 방식 변경
    quality = len(matches) / num_keypoints

    return quality, avg_distance, len(matches)

def evaluate_repeatability(keypoints1, keypoints2, matches, shape):
    if len(matches) < 4:
        return 0

    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    try:
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        if H is None:
            return 0

        h, w = shape
        pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, H)

        good_matches = mask.ravel() == 1
        return np.sum(good_matches) / len(matches)

    except cv2.error:
        return 0

def preprocess_image(image):
    # 가우시안 블러 적용
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # 샤프닝
    sharpened = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    return sharpened

def evaluate_algorithm_extended(algorithm, image_folder):
    image_files = sorted(os.listdir(image_folder))
    quality_scores = []
    repeatability_scores = []
    total_keypoints = []
    match_counts = []
    avg_distances = []

    for i in tqdm(range(len(image_files) - 1), desc=f"Evaluating {algorithm}"):
        image1_path = os.path.join(image_folder, image_files[i])
        image2_path = os.path.join(image_folder, image_files[i+1])

        image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
        image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

        if image1 is None or image2 is None:
            print(f"Warning: Unable to read image {image_files[i]} or {image_files[i+1]}")
            continue

        # 이미지 전처리 적용
        image1 = preprocess_image(image1)
        image2 = preprocess_image(image2)

        keypoints1, descriptors1 = detect_features(image1, algorithm)
        keypoints2, descriptors2 = detect_features(image2, algorithm)

        total_keypoints.append(len(keypoints1))

        matches = match_features(descriptors1, descriptors2, algorithm)
        match_counts.append(len(matches))

        quality_score, avg_distance, good_match_count = evaluate_feature_quality(matches, len(keypoints1))
        quality_scores.append(quality_score)
        avg_distances.append(avg_distance)

        repeatability_score = evaluate_repeatability(keypoints1, keypoints2, matches, image1.shape)
        repeatability_scores.append(repeatability_score)

    avg_keypoints = np.mean(total_keypoints) if total_keypoints else 0
    avg_matches = np.mean(match_counts) if match_counts else 0
    avg_quality = np.mean(quality_scores) if quality_scores else 0
    avg_repeatability = np.mean(repeatability_scores) if repeatability_scores else 0
    avg_distance = np.mean(avg_distances) if avg_distances else 0

    return avg_quality, avg_repeatability, avg_keypoints, avg_matches, avg_distance

# 메인 실행 부분
image_folder = "/content/drive/MyDrive/KeyPoint/dataset_split/test/damaged"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

extended_results = {}

for algorithm in algorithms:
    quality, repeatability, avg_keypoints, avg_matches, avg_distance = evaluate_algorithm_extended(algorithm, image_folder)
    extended_results[algorithm] = {
        'quality': quality,
        'repeatability': repeatability,
        'avg_keypoints': avg_keypoints,
        'avg_matches': avg_matches,
        'avg_distance': avg_distance
    }

print("Extended Results:")
for alg, res in extended_results.items():
    print(f"{alg}:")
    print(f"  Quality = {res['quality']:.4f}")
    print(f"  Repeatability = {res['repeatability']:.4f}")
    print(f"  Avg Keypoints = {res['avg_keypoints']:.2f}")
    print(f"  Avg Matches = {res['avg_matches']:.2f}")
    print(f"  Avg Match Distance = {res['avg_distance']:.2f}")
    print()

Evaluating BRISK: 100%|██████████| 59/59 [01:07<00:00,  1.14s/it]

Extended Results:
SIFT:
  Quality = 0.2130
  Repeatability = 0.1480
  Avg Keypoints = 283.42
  Avg Matches = 56.42
  Avg Match Distance = 313.58

ORB:
  Quality = 0.1967
  Repeatability = 0.0744
  Avg Keypoints = 770.39
  Avg Matches = 151.68
  Avg Match Distance = 63.36

AKAZE:
  Quality = 0.2481
  Repeatability = 0.2063
  Avg Keypoints = 180.76
  Avg Matches = 38.97
  Avg Match Distance = 603.13

BRISK:
  Quality = 0.2384
  Repeatability = 0.0361
  Avg Keypoints = 2514.49
  Avg Matches = 495.34
  Avg Match Distance = 625.42



In [8]:
import cv2
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

def detect_features(image, algorithm):
    if algorithm == 'SIFT':
        detector = cv2.SIFT_create(nfeatures=500, contrastThreshold=0.04)
    elif algorithm == 'ORB':
        detector = cv2.ORB_create(nfeatures=1000, scaleFactor=1.2, nlevels=8)
    elif algorithm == 'AKAZE':
        detector = cv2.AKAZE_create(threshold=0.001)
    elif algorithm == 'BRISK':
        detector = cv2.BRISK_create(thresh=30, octaves=3)
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")

    keypoints, descriptors = detector.detectAndCompute(image, None)
    return keypoints, descriptors

def match_features(descriptors1, descriptors2, algorithm):
    if descriptors1 is None or descriptors2 is None:
        return []

    if algorithm in ['SIFT', 'AKAZE', 'BRISK']:
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    else:  # ORB
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # 상위 75%의 매치만 사용
    good_matches = matches[:int(len(matches) * 0.75)]

    return good_matches

def preprocess_image(image):
    # 가우시안 블러 적용
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # 샤프닝
    sharpened = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    return sharpened

def visualize_features_and_matches(image1, image2, keypoints1, keypoints2, matches, algorithm):
    # 특징점 시각화
    img1_keypoints = cv2.drawKeypoints(image1, keypoints1, None, color=(0, 255, 0))
    img2_keypoints = cv2.drawKeypoints(image2, keypoints2, None, color=(0, 255, 0))

    # 매칭 결과 시각화
    img_matches = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches, None,
                                  flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    # 결과 출력
    plt.figure(figsize=(20, 10))
    plt.subplot(221), plt.imshow(cv2.cvtColor(img1_keypoints, cv2.COLOR_BGR2RGB))
    plt.title(f'{algorithm} Keypoints - Image 1'), plt.axis('off')
    plt.subplot(222), plt.imshow(cv2.cvtColor(img2_keypoints, cv2.COLOR_BGR2RGB))
    plt.title(f'{algorithm} Keypoints - Image 2'), plt.axis('off')
    plt.subplot(212), plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.title(f'{algorithm} Matches'), plt.axis('off')
    plt.tight_layout()
    plt.show()

def process_image_pair(image1_path, image2_path, algorithm):
    # 이미지 읽기
    image1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)

    if image1 is None or image2 is None:
        print(f"Warning: Unable to read image {image1_path} or {image2_path}")
        return

    # 이미지 전처리
    image1 = preprocess_image(image1)
    image2 = preprocess_image(image2)

    # 특징점 검출
    keypoints1, descriptors1 = detect_features(image1, algorithm)
    keypoints2, descriptors2 = detect_features(image2, algorithm)

    # 특징점 매칭
    matches = match_features(descriptors1, descriptors2, algorithm)

    # 결과 시각화
    visualize_features_and_matches(image1, image2, keypoints1, keypoints2, matches, algorithm)

    return len(keypoints1), len(keypoints2), len(matches)

# 메인 실행 부분
image_folder = "/content/drive/MyDrive/KeyPoint/dataset_split/test/damaged"
algorithms = ['SIFT', 'ORB', 'AKAZE', 'BRISK']

# 첫 두 개의 이미지에 대해 모든 알고리즘 실행
image_files = sorted(os.listdir(image_folder))
image1_path = os.path.join(image_folder, image_files[0])
image2_path = os.path.join(image_folder, image_files[1])

for algorithm in algorithms:
    print(f"\nProcessing {algorithm}:")
    num_keypoints1, num_keypoints2, num_matches = process_image_pair(image1_path, image2_path, algorithm)
    print(f"Number of Keypoints in Image 1: {num_keypoints1}")
    print(f"Number of Keypoints in Image 2: {num_keypoints2}")
    print(f"Number of Matches: {num_matches}")

Output hidden; open in https://colab.research.google.com to view.

각 알고리즘의 성능을 분석

1. SIFT:
   - 중간 정도의 특징점 수를 검출합니다.
   - 두 이미지 간 특징점 수의 차이가 있습니다 (240 vs 404).
   - 60개의 매치를 찾았습니다. 첫 번째 이미지의 특징점 중 25%가 매치되었습니다.

2. ORB:
   - SIFT보다 더 많은 특징점을 검출합니다.
   - 두 이미지에서 비슷한 수의 특징점을 찾았습니다 (782 vs 868).
   - 128개의 매치로, 가장 많은 매치를 찾았습니다. 첫 번째 이미지의 특징점 중 약 16.4%가 매치되었습니다.

3. AKAZE:
   - 가장 적은 수의 특징점을 검출합니다.
   - 두 이미지 간 특징점 수의 차이가 큽니다 (79 vs 238).
   - 20개의 매치로, 가장 적은 매치를 찾았습니다. 하지만 첫 번째 이미지의 특징점 중 약 25.3%가 매치되어 SIFT와 비슷한 비율을 보입니다.

4. BRISK:
   - 가장 많은 특징점을 검출합니다.
   - 두 이미지에서 모두 3000개 이상의 특징점을 찾았습니다.
   - 759개의 매치로, 절대적인 수치로는 가장 많은 매치를 찾았습니다. 하지만 첫 번째 이미지의 특징점 중 약 21.2%가 매치되어 비율로는 중간 정도입니다.

분석:

1. 특징점 검출 능력:
   BRISK > ORB > SIFT > AKAZE
   BRISK가 압도적으로 많은 특징점을 찾지만, 이것이 항상 좋은 것은 아닙니다. 처리 시간이 길어질 수 있습니다.

2. 매칭 수:
   BRISK > ORB > SIFT > AKAZE
   절대적인 매치 수에서는 BRISK가 가장 뛰어납니다.

3. 매칭 비율 (첫 번째 이미지의 특징점 대비):
   AKAZE (25.3%) ≈ SIFT (25%) > BRISK (21.2%) > ORB (16.4%)
   AKAZE와 SIFT가 가장 높은 매칭 비율을 보입니다.

4. 일관성:
   ORB가 두 이미지에서 가장 일관된 수의 특징점을 찾았습니다.

결론:
1. 정확성이 가장 중요하다면 AKAZE나 SIFT를 선택하는 것이 좋습니다. 이들은 높은 매칭 비율을 보입니다.
2. 많은 특징점과 매치가 필요하다면 BRISK가 좋은 선택일 수 있습니다.
3. 속도와 일관성이 중요하다면 ORB를 고려해볼 만합니다.
